## Quantum Measurement Classification with Pure States on Ten Classes MNIST with quantum-enhanced Fourier features

Diego Useche

## GPU

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Libraries

In [ ]:

!pip install tensorcircuit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 13.4 MB/s eta 0:00:00


In [ ]:
from functools import partial
import numpy as np
import tensorflow as tf

import tensorcircuit as tc
from tensorcircuit import keras

In [ ]:
tc.set_backend("tensorflow")
tc.set_dtype("complex128")

('complex128', 'float64')

## Utils functions

In [ ]:
# this function takes the number of classes and of qubits of the qmc pure, and extract the indices
# of the bit strings that correpond to the classes prediction
## Example, qmc prediction bit string ["00", "01", "10", "11"]
## the classes prediction is encoded in ["00", "10"], then it returns [0, 2]

def indices_qubits_clases(num_qubits_param, num_classes_param):
  num_qubits_classes_temp = int(np.ceil(np.log2(num_classes_param)))
  a = [np.binary_repr(i, num_qubits_param) for i in range(2**num_qubits_param)]
  b = [(np.binary_repr(i, num_qubits_classes_temp) + "0"*(num_qubits_param - num_qubits_classes_temp)) for i in range(num_classes_param)]
  indices_temp = []
  for i in range(len(a)):
    if a[i] in b:
      indices_temp.append(i)

  return indices_temp

indices_qubits_clases(4, 4)

[0, 4, 8, 12]

## MNIST Data Set

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_test = X_train[..., np.newaxis]/255.0, X_test[..., np.newaxis]/255.0

# Resize images to 4x4
img_compress_size = 8
X_train = tf.image.resize(X_train, (img_compress_size, img_compress_size)).numpy().reshape(-1, img_compress_size, img_compress_size)
X_test = tf.image.resize(X_test, (img_compress_size , img_compress_size)).numpy().reshape(-1, img_compress_size, img_compress_size)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

11490434/11490434 [==============================] - 0s 0us/step


((60000, 8, 8), (10000, 8, 8), (60000,), (10000,))

In [ ]:
# Select the indices for the binary classification
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_oh = tf.reshape (tf.keras.backend.one_hot(y_train, 10), (-1,10))
y_test_oh = tf.reshape (tf.keras.backend.one_hot(y_test, 10), (-1,10))

# reshape the X_train_n , and X_test_n,
n_train = X_train.shape[0]
n_test = X_test.shape[0]
l_side = X_test.shape[1]
X_train = X_train.reshape((n_train, l_side**2))
X_test = X_test.reshape((n_test, l_side**2))

X_train.shape, X_test.shape, y_train_oh.shape, y_test_oh.shape

((60000, 64), (10000, 64), TensorShape([60000, 10]), TensorShape([10000, 10]))

## Model 1: Mixed QMC, QEFF, No-conv layer

Ten classes MNIST Classification, QMC variational, with quantum-enhanced Fourier features, no-conv layer

In [ ]:
# this function takes the number of classes and of qubits of the qmc pure, and extract the indices
# of the bit strings that correpond to the classes prediction
def indices_qubits_clases(num_qubits_param, num_classes_param):
  num_qubits_classes_temp = int(np.ceil(np.log2(num_classes_param)))
  a = [np.binary_repr(i, num_qubits_param) for i in range(2**num_qubits_param)]
  b = [(np.binary_repr(i, num_qubits_classes_temp) + "0"*(num_qubits_param - num_qubits_classes_temp)) for i in range(num_classes_param)]
  indices_temp = []
  for i in range(len(a)):
    if a[i] in b:
      indices_temp.append(i)

  return indices_temp

indices_qubits_clases(6, 10)

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36]

In [ ]:
### Quantum variational KDC with QEFF

import tensorcircuit as tc
from tensorcircuit import keras
import tensorflow as tf

from functools import partial
import numpy as np
import math as m
from scipy.stats import entropy, spearmanr



tc.set_backend("tensorflow")
tc.set_dtype("complex128")

pi = tf.constant(m.pi)


class QVKDC_MIXED_QEFF:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, qeff_weights_param, num_classes_param, auto_compile=True, var_pure_state_size=64, gamma=2., epochs_param=15):

        self.circuit = None
        self.gamma = gamma
        self.num_classes = num_classes_param
        self.var_pure_state_parameters_size = 2*var_pure_state_size - 2
        self.qeff_weights_param =  qeff_weights_param
        self.epochs = epochs_param

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_pure_state_parameters_size,)]
            )

        self.model = tf.keras.Sequential([layer])

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_pure_state_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        n_total_qubits_temp = int(np.log2((len(var_pure_state_param)+2)/2))
        n_qeffs_temp = int((self.qeff_weights_param.shape[1] + 2)/2)
        n_qubits_classes_qeff_temp = int(np.ceil(np.log2((self.num_classes))+np.ceil(np.log2((n_qeffs_temp)))))
        n_qeff_qubits_temp = int(np.ceil(np.log2((n_qeffs_temp))))
        n_classes_qubits_temp = int(np.ceil(np.log2(self.num_classes)))

        ### iterators qeff, ps
        index_iter_qeff = iter(np.arange(self.qeff_weights_param.shape[1]))
        index_iter_ps  = iter(np.arange(len(var_pure_state_param)))

        ### indices classes, of ms
        index_qubit_states = indices_qubits_clases(n_qubits_classes_qeff_temp, self.num_classes) # extract indices of the bit string of classes

        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(n_total_qubits_temp)

        def circuit_base_ry_n(qc_param, num_qubits_param, target_qubit_param):
            if num_qubits_param == 1:
                qc_param.ry(0, theta = var_pure_state_param[next(index_iter_ps)])
            elif num_qubits_param == 2:
                qc_param.ry(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                qc_param.cnot(0, target_qubit_param)
                qc_param.ry(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                return
            else:
                circuit_base_ry_n(qc_param, num_qubits_param-1, target_qubit_param)
                qc_param.cnot(num_qubits_param-2, target_qubit_param)
                circuit_base_ry_n(qc_param, num_qubits_param-1, target_qubit_param)
                target_qubit_param -= 1

        def circuit_base_rz_n(qc_param, num_qubits_param, target_qubit_param):
            if num_qubits_param == 1:
                qc_param.rz(0, theta = var_pure_state_param[next(index_iter_ps)])
            elif num_qubits_param == 2:
                qc_param.rz(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                qc_param.cnot(0, target_qubit_param)
                qc_param.rz(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                return
            else:
                circuit_base_rz_n(qc_param, num_qubits_param-1, target_qubit_param)
                qc_param.cnot(num_qubits_param-2, target_qubit_param)
                circuit_base_rz_n(qc_param, num_qubits_param-1, target_qubit_param)
                target_qubit_param -= 1

        # Learning pure state
        for i in range(1, n_total_qubits_temp+1):
            circuit_base_ry_n(self.circuit, i, i-1)

        # Learning pure state complex phase
        for j in range(1, n_total_qubits_temp+1):
            circuit_base_rz_n(self.circuit, j, j-1)

        # Value to predict

        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        phases_temp = tf.cast(tf.sqrt(self.gamma), tf.float64)*tf.linalg.matmul(tf.cast(x_sample_temp, tf.float64), self.qeff_weights_param)
        init_qubit_qeff_temp = n_classes_qubits_temp # qubit at which the qaff mapping starts it starts after the qubits of the classes

        def circuit_base_rz_qeff_n(qc_param, num_qubits_param, target_qubit_param, init_qubit_param):
          if num_qubits_param == 1:
            qc_param.rz(init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(init_qubit_param)
            qc_param.rz(init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
          elif num_qubits_param == 2:
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.cnot(init_qubit_param, target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            return
          else:
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            qc_param.cnot(num_qubits_param-2+init_qubit_param, target_qubit_param+init_qubit_param)
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            target_qubit_param -= 1

        # Applying the QEFF feature map

        for i in range(n_qubits_classes_qeff_temp - init_qubit_qeff_temp + 1 - 1, 1 - 1, -1):
          circuit_base_rz_qeff_n(self.circuit, i, i - 1, init_qubit_qeff_temp)

        for i in range(init_qubit_qeff_temp, n_qubits_classes_qeff_temp):
          self.circuit.H(i)

        # Trace out ancilla qubits, find probability of |k> state for the prediction of class k

        measurement_state = tc.quantum.reduced_density_matrix(self.circuit.state(), cut=[m for m in range(n_qubits_classes_qeff_temp, n_total_qubits_temp)])
        measurements_results = (1./(self.epochs))*tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        return measurements_results

    def compile(
            self,
            optimizer=tf.keras.optimizers.legacy.Adam(0.0005), # originally 0.0005
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = "categorical_crossentropy",
            optimizer=optimizer,
            metrics=["accuracy"],
            **kwargs
        )
    def fit(self, x_train, y_train, batch_size=16, **kwargs):
        r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size=batch_size, epochs=self.epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      print(x_test.shape[1])
      return (tf.power((self.gamma/(pi)), x_test.shape[1])*\
          self.model.predict(x_test)).numpy()


In [ ]:
N_QEFFS = 8
NUM_CLASSES = 10
dim_x = 8**2 # 8x8 images
GAMMA = 2**(-2) # originally 2**(-4)
num_data = X_train.shape[0]
EPOCHS = 10
PURE_STATE_SIZE = 256

weights_qeff_method = np.random.normal(size=(X_train.shape[1], 2*(N_QEFFS-1)))/np.sqrt((N_QEFFS-1))

weights_qeff_method.mean(), weights_qeff_method.std(), weights_qeff_method.shape, num_data

(-0.0003616293817530976, 0.3825734149174446, (64, 14), 60000)

In [ ]:
qvkdc = QVKDC_MIXED_QEFF(qeff_weights_param = weights_qeff_method, num_classes_param = NUM_CLASSES, var_pure_state_size=PURE_STATE_SIZE, gamma=GAMMA, epochs_param = EPOCHS)

qvkdc.fit(X_train, y_train_oh, batch_size=16)

Epoch 1/10
3750/3750 [==============================] - 507s 67ms/step - loss: 2.9741 - accuracy: 0.1977
Epoch 2/10
3750/3750 [==============================] - 211s 56ms/step - loss: 1.7899 - accuracy: 0.3885
Epoch 3/10
3750/3750 [==============================] - 268s 71ms/step - loss: 1.7135 - accuracy: 0.4204
Epoch 4/10
3750/3750 [==============================] - 287s 77ms/step - loss: 1.6983 - accuracy: 0.4268
Epoch 5/10
 276/3750 [=>............................] - ETA: 3:09 - loss: 1.6998 - accuracy: 0.4198

KeyboardInterrupt: 

## Model 2: Mixed QMC variational, QEFF, with Le-Net Conv layer

### Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD

print(tf.__version__)
print(tf.config.list_logical_devices())

2.15.0
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


### MNIST Data Set

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_test = X_train[..., np.newaxis]/255.0, X_test[..., np.newaxis]/255.0

X_train.shape, X_test.shape, y_train.shape, y_test.shape

11490434/11490434 [==============================] - 0s 0us/step


((60000, 28, 28, 1), (10000, 28, 28, 1), (60000,), (10000,))

In [ ]:
# Select the indices for the binary classification
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_oh = tf.reshape (tf.keras.backend.one_hot(y_train, 10), (-1,10))
y_test_oh = tf.reshape (tf.keras.backend.one_hot(y_test, 10), (-1,10))

X_train.shape, X_test.shape, y_train_oh.shape, y_test_oh.shape

((60000, 28, 28, 1),
 (10000, 28, 28, 1),
 TensorShape([60000, 10]),
 TensorShape([10000, 10]))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD

print(tf.__version__)
print(tf.config.list_logical_devices())

2.15.0
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


In [ ]:
### Quantum variational KDC with QEFF

import tensorcircuit as tc
from tensorcircuit import keras
import tensorflow as tf

from functools import partial
import numpy as np
import math as m
from scipy.stats import entropy, spearmanr



tc.set_backend("tensorflow")
tc.set_dtype("complex128")

pi = tf.constant(m.pi)


class QVKDC_MIXED_QEFF_LENET:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, qeff_weights_param, num_classes_param, input_dim_param, dim_lenet_out_param, auto_compile=True, var_pure_state_size=64, gamma=2., epochs_param=15):

        self.circuit = None
        self.gamma = gamma
        self.num_classes = num_classes_param
        self.var_pure_state_parameters_size = 2*var_pure_state_size - 2
        self.qeff_weights_param =  qeff_weights_param
        self.epochs = epochs_param
        self.dim_lenet_out = dim_lenet_out_param

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_pure_state_parameters_size,)]
            )

        ## build the Let-net model
        self.model = Sequential()
        self.model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(input_dim_param, input_dim_param, 1)))
        self.model.add(tf.keras.layers.AveragePooling2D())
        self.model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu'))
        self.model.add(tf.keras.layers.AveragePooling2D())
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=84, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=self.dim_lenet_out, activation = 'relu')) # original "softmax"

        # add the quantum layer

        self.model.add(layer)

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_pure_state_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        n_total_qubits_temp = int(np.log2((len(var_pure_state_param)+2)/2))
        n_qeffs_temp = int((self.qeff_weights_param.shape[1] + 2)/2)
        n_qubits_classes_qeff_temp = int(np.ceil(np.log2((self.num_classes))+np.ceil(np.log2((n_qeffs_temp)))))
        n_qeff_qubits_temp = int(np.ceil(np.log2((n_qeffs_temp))))
        n_classes_qubits_temp = int(np.ceil(np.log2(self.num_classes)))

        ### iterators qeff, ps
        index_iter_qeff = iter(np.arange(self.qeff_weights_param.shape[1]))
        index_iter_ps  = iter(np.arange(len(var_pure_state_param)))

        ### indices classes, of ms
        index_qubit_states = indices_qubits_clases(n_qubits_classes_qeff_temp, self.num_classes) # extract indices of the bit string of classes

        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(n_total_qubits_temp)

        def circuit_base_ry_n(qc_param, num_qubits_param, target_qubit_param):
            if num_qubits_param == 1:
                qc_param.ry(0, theta = var_pure_state_param[next(index_iter_ps)])
            elif num_qubits_param == 2:
                qc_param.ry(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                qc_param.cnot(0, target_qubit_param)
                qc_param.ry(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                return
            else:
                circuit_base_ry_n(qc_param, num_qubits_param-1, target_qubit_param)
                qc_param.cnot(num_qubits_param-2, target_qubit_param)
                circuit_base_ry_n(qc_param, num_qubits_param-1, target_qubit_param)
                target_qubit_param -= 1

        def circuit_base_rz_n(qc_param, num_qubits_param, target_qubit_param):
            if num_qubits_param == 1:
                qc_param.rz(0, theta = var_pure_state_param[next(index_iter_ps)])
            elif num_qubits_param == 2:
                qc_param.rz(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                qc_param.cnot(0, target_qubit_param)
                qc_param.rz(target_qubit_param, theta=var_pure_state_param[next(index_iter_ps)])
                return
            else:
                circuit_base_rz_n(qc_param, num_qubits_param-1, target_qubit_param)
                qc_param.cnot(num_qubits_param-2, target_qubit_param)
                circuit_base_rz_n(qc_param, num_qubits_param-1, target_qubit_param)
                target_qubit_param -= 1

        # Learning pure state
        for i in range(1, n_total_qubits_temp+1):
            circuit_base_ry_n(self.circuit, i, i-1)

        # Learning pure state complex phase
        for j in range(1, n_total_qubits_temp+1):
            circuit_base_rz_n(self.circuit, j, j-1)

        # Value to predict

        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        phases_temp = tf.sqrt(tf.cast(self.gamma, tf.float64))*tf.linalg.matmul(tf.cast(x_sample_temp, tf.float64), self.qeff_weights_param)
        init_qubit_qeff_temp = n_classes_qubits_temp # qubit at which the qaff mapping starts it starts after the qubits of the classes

        def circuit_base_rz_qeff_n(qc_param, num_qubits_param, target_qubit_param, init_qubit_param):
          if num_qubits_param == 1:
            qc_param.rz(init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(init_qubit_param)
            qc_param.rz(init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
          elif num_qubits_param == 2:
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.cnot(init_qubit_param, target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            return
          else:
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            qc_param.cnot(num_qubits_param-2+init_qubit_param, target_qubit_param+init_qubit_param)
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            target_qubit_param -= 1

        # Applying the QEFF feature map

        for i in range(n_qubits_classes_qeff_temp - init_qubit_qeff_temp + 1 - 1, 1 - 1, -1):
          circuit_base_rz_qeff_n(self.circuit, i, i - 1, init_qubit_qeff_temp)

        for i in range(init_qubit_qeff_temp, n_qubits_classes_qeff_temp):
          self.circuit.H(i)

        # Trace out ancilla qubits, find probability of |k> state for the prediction of class k

        measurement_state = tc.quantum.reduced_density_matrix(self.circuit.state(), cut=[m for m in range(n_qubits_classes_qeff_temp, n_total_qubits_temp)])
        measurements_results = (1./(self.epochs))*tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        return measurements_results

    def compile(
            self,
            optimizer=tf.keras.optimizers.legacy.Adam(0.0005), # originally 0.0005
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = "categorical_crossentropy",
            optimizer=optimizer,
            metrics=["accuracy"],
            **kwargs
        )
    def fit(self, x_train, y_train, batch_size=16, **kwargs):
 with         r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size=batch_size, epochs=self.epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      return (tf.math.pow((self.gamma/(pi)), self.dim_lenet_out)*\
          self.model.predict(x_test)).numpy()

In [ ]:
# Define a LeNet CNN feature extraction model
input_shape = X_train.shape[1:]
INPUT_DIM = input_shape[0]
N_QEFFS = 16
NUM_CLASSES = 10
DIM_LENET_OUT = 14 # originally 16
GAMMA = 2**(0)
EPOCHS = 10
PURE_STATE_SIZE = 512

weights_qeff_method = np.random.normal(size=(DIM_LENET_OUT, 2*(N_QEFFS-1)))/np.sqrt((N_QEFFS-1))

weights_qeff_method.mean(), weights_qeff_method.std(), weights_qeff_method.shape

(0.002734544694931644, 0.2563007536159625, (14, 30))

In [ ]:
qvkdc_lenet = QVKDC_MIXED_QEFF_LENET(qeff_weights_param = weights_qeff_method, num_classes_param = NUM_CLASSES, dim_lenet_out_param = DIM_LENET_OUT, input_dim_param = INPUT_DIM, var_pure_state_size=PURE_STATE_SIZE, gamma=GAMMA, epochs_param = EPOCHS)
qvkdc_lenet.fit(X_train, y_train_oh, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 1262s 382ms/step - loss: 2.4926 - accuracy: 0.2635
Epoch 2/10
1875/1875 [==============================] - 702s 374ms/step - loss: 1.4787 - accuracy: 0.4387
Epoch 3/10
1875/1875 [==============================] - 703s 375ms/step - loss: 1.1034 - accuracy: 0.6011
Epoch 4/10
1875/1875 [==============================] - 706s 377ms/step - loss: 0.7970 - accuracy: 0.7467
Epoch 5/10
1875/1875 [==============================] - 702s 374ms/step - loss: 0.6683 - accuracy: 0.7717
Epoch 6/10
1875/1875 [==============================] - 701s 374ms/step - loss: 0.5621 - accuracy: 0.7789
Epoch 7/10
1875/1875 [==============================] - 696s 371ms/step - loss: 0.4405 - accuracy: 0.8011
Epoch 8/10
1875/1875 [==============================] - 694s 370ms/step - loss: 0.4145 - accuracy: 0.8303
Epoch 9/10
1875/1875 [==============================] - 693s 370ms/step - loss: 0.3732 - accuracy: 0.8763
Epoch 10/10
1875/1875 [======================

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = qvkdc_lenet.predict(X_test)
accuracy_score(y_test, np.argmax(y_pred, axis=1))

313/313 [==============================] - 450s 78ms/step


0.8913

## Model 3: Mixed QMC variational, QEFF, with Le-Net Conv layer with HEA



#### Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD

print(tf.__version__)
print(tf.config.list_logical_devices())

2.15.0
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


#### MNIST Data Set

In [ ]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_test = X_train[..., np.newaxis]/255.0, X_test[..., np.newaxis]/255.0

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1), (60000,), (10000,))

In [ ]:
# Select the indices for the binary classification
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_oh = tf.reshape (tf.keras.backend.one_hot(y_train, 10), (-1,10))
y_test_oh = tf.reshape (tf.keras.backend.one_hot(y_test, 10), (-1,10))

X_train.shape, X_test.shape, y_train_oh.shape, y_test_oh.shape

((60000, 28, 28, 1),
 (10000, 28, 28, 1),
 TensorShape([60000, 10]),
 TensorShape([10000, 10]))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD

print(tf.__version__)
print(tf.config.list_logical_devices())

2.15.0
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]


The number of parameters of the Hardware efficient ansatz is given by,


```
HEA_size = n_qubits * (num_layers_hea + 1) * 2
```



In [ ]:
### Quantum variational KDC with QEFF

import tensorcircuit as tc
from tensorcircuit import keras
import tensorflow as tf

from functools import partial
import numpy as np
import math as m
from scipy.stats import entropy, spearmanr



tc.set_backend("tensorflow")
tc.set_dtype("complex128")

pi = tf.constant(m.pi)


class QVKDC_MIXED_QEFF_LENET_HEA:
    r"""
    Defines the ready-to-use Quantum measurement classification (QMC) model implemented
    in TensorCircuit using the TensorFlow/Keras API. Any additional argument in the methods has to be Keras-compliant.

    Args:
        auto_compile: A boolean to autocompile the model using default settings. (Default True).
        var_pure_state_size:
        gamma:

    Returns:
        An instantiated model ready to train with ad-hoc data.

    """
    def __init__(self, qeff_weights_param, num_classes_param, input_dim_param, dim_lenet_out_param, auto_compile=True, var_hea_ansatz_size_param=64, num_layers_hea_param = 3, gamma=2., epochs_param=15):

        self.circuit = None
        self.gamma = gamma
        self.num_classes = num_classes_param
        self.num_layers_hea = num_layers_hea_param
        self.var_hea_ansatz_size = var_hea_ansatz_size_param
        self.qeff_weights_param =  qeff_weights_param
        self.epochs = epochs_param
        self.dim_lenet_out = dim_lenet_out_param

        layer = keras.QuantumLayer(
            partial(self.layer),
            [(self.var_hea_ansatz_size,)]
            )

        ## build the Let-net model
        self.model = Sequential()
        self.model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(input_dim_param, input_dim_param, 1)))
        self.model.add(tf.keras.layers.AveragePooling2D())
        self.model.add(tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu'))
        self.model.add(tf.keras.layers.AveragePooling2D())
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=84, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=self.dim_lenet_out, activation = 'relu')) # original "softmax"

        # add the quantum layer

        self.model.add(layer)
        print(self.model.summary())

        if auto_compile:
            self.compile()

    def layer(
            self,
            x_sample_param,
            var_hea_ansatz_param,
        ):
        r"""
        Defines a Density Matrix Kernel Density Estimation quantum layer for learning with fixed qaff (Meaning of qaff?). (This function was originally named dmkde_mixed_variational_density_estimation_fixed_qaff)

        Args:
            U_dagger:
            var_pure_state_param:

        Returns:
            The probabilities of :math:`|k\rangle`, `|1\rangle`, ..., `|k\rangle` state for kernel density classification of the classes.
        """

        n_total_qubits_temp = int(self.var_hea_ansatz_size/((self.num_layers_hea+1)*2))
        n_qeffs_temp = int((self.qeff_weights_param.shape[1] + 2)/2)
        n_qubits_classes_qeff_temp = int(np.ceil(np.log2((self.num_classes))+np.ceil(np.log2((n_qeffs_temp)))))
        n_qeff_qubits_temp = int(np.ceil(np.log2((n_qeffs_temp))))
        n_classes_qubits_temp = int(np.ceil(np.log2(self.num_classes)))

        #print(n_total_qubits_temp, n_qeffs_temp, n_qubits_classes_qeff_temp, n_qeff_qubits_temp, n_classes_qubits_temp)
        ### iterators qeff, ps
        index_iter_qeff = iter(np.arange(self.qeff_weights_param.shape[1]))
        index_iter_hea  = iter(np.arange(len(var_hea_ansatz_param)))

        ### indices classes, of ms
        index_qubit_states = indices_qubits_clases(n_qubits_classes_qeff_temp, self.num_classes) # extract indices of the bit string of classes

        # Instantiate a circuit with the calculated number of qubits.
        self.circuit = tc.Circuit(n_total_qubits_temp)

        def hea_ansatz(qc_param, num_qubits_param, num_layers_param):
        # encoding
          for i in range (0, num_qubits_param):
            qc_param.ry(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
            qc_param.rz(i, theta = var_hea_ansatz_param[next(index_iter_hea)])
        # layers
          for j in range(num_layers_param):
            for i in range (0, num_qubits_param-1):
              qc_param.CNOT(i, i+1)

            for i in range (0, num_qubits_param):
              qc_param.ry(i, theta= var_hea_ansatz_param[next(index_iter_hea)])
              qc_param.rz(i, theta= var_hea_ansatz_param[next(index_iter_hea)])

        ## learning pure state with HEA

        hea_ansatz(self.circuit, n_total_qubits_temp, self.num_layers_hea)

        # Value to predict

        x_sample_temp = tf.expand_dims(x_sample_param, axis=0)
        phases_temp = tf.sqrt(tf.cast(self.gamma, tf.float64))*tf.linalg.matmul(tf.cast(x_sample_temp, tf.float64), self.qeff_weights_param)
        init_qubit_qeff_temp = n_classes_qubits_temp # qubit at which the qaff mapping starts it starts after the qubits of the classes

        def circuit_base_rz_qeff_n(qc_param, num_qubits_param, target_qubit_param, init_qubit_param):
          if num_qubits_param == 1:
            qc_param.rz(init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(init_qubit_param)
            qc_param.rz(init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
          elif num_qubits_param == 2:
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.cnot(init_qubit_param, target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            qc_param.X(target_qubit_param + init_qubit_param)
            qc_param.rz(target_qubit_param + init_qubit_param, theta = phases_temp[0][next(index_iter_qeff)])
            return
          else:
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            qc_param.cnot(num_qubits_param-2+init_qubit_param, target_qubit_param+init_qubit_param)
            circuit_base_rz_qeff_n(qc_param, num_qubits_param-1, target_qubit_param, init_qubit_param)
            target_qubit_param -= 1

        # Applying the QEFF feature map

        for i in range(n_qubits_classes_qeff_temp - init_qubit_qeff_temp + 1 - 1, 1 - 1, -1):
          circuit_base_rz_qeff_n(self.circuit, i, i - 1, init_qubit_qeff_temp)

        for i in range(init_qubit_qeff_temp, n_qubits_classes_qeff_temp):
          self.circuit.H(i)

        # Trace out ancilla qubits, find probability of |k> state for the prediction of class k

        measurement_state = tc.quantum.reduced_density_matrix(self.circuit.state(), cut=[m for m in range(n_qubits_classes_qeff_temp, n_total_qubits_temp)])
        measurements_results = (1./(self.epochs))*tc.backend.real(tf.stack([measurement_state[index_qubit_states[i], index_qubit_states[i]] for i in range(self.num_classes)]))
        return measurements_results

    def compile(
            self,
            optimizer=tf.keras.optimizers.legacy.Adam(0.0005), # originally 0.0005
            **kwargs):
        r"""
        Method to compile the model.

        Args:
            optimizer:
            **kwargs: Any additional argument.

        Returns:
            None.
        """
        self.model.compile(
            loss = "categorical_crossentropy",
            optimizer=optimizer,
            metrics=["accuracy"],
            **kwargs
        )
    def fit(self, x_train, y_train, batch_size=16, **kwargs):
        r"""
        Method to fit (train) the model using the ad-hoc dataset.

        Args:
            x_train:
            y_train:
            batch_size:
            epochs:
            **kwargs: Any additional argument.

        Returns:
            None.
        """

        self.model.fit(x_train, y_train, batch_size=batch_size, epochs=self.epochs, **kwargs)

    def predict(self, x_test):
      r"""
      Method to make predictions with the trained model.

      Args:
          x_test:

      Returns:
          The predictions of the conditional density estimation of the input data.
      """
      return (tf.math.pow((self.gamma/(pi)), self.dim_lenet_out/2)*\
          self.model.predict(x_test)).numpy()

In [ ]:
# Define a LeNet CNN feature extraction model
input_shape = X_train.shape[1:]
INPUT_DIM = input_shape[0]
N_QEFFS = 16
NUM_CLASSES = 10
DIM_LENET_OUT = 14 # originally 16
GAMMA = 2**(0)
EPOCHS = 10
NUM_LAYERS_HEA = 3
NUM_ANCILLA_QUBITS = 1
NUM_TOTAL_QUBITS = int(np.ceil(np.log2(NUM_CLASSES)) + np.ceil(np.log2(N_QEFFS)) + NUM_ANCILLA_QUBITS)
HEA_ANSATZ_SIZE = int(NUM_TOTAL_QUBITS*(NUM_LAYERS_HEA+1)*2)

weights_qeff_method = np.random.normal(size=(DIM_LENET_OUT, 2*(N_QEFFS-1)))/np.sqrt((N_QEFFS-1))

weights_qeff_method.mean(), weights_qeff_method.std(), weights_qeff_method.shape, HEA_ANSATZ_SIZE, NUM_TOTAL_QUBITS

(0.0011090572287938744, 0.2598442347195675, (14, 30), 72, 9)

In [ ]:
qvkdc_hea_lenet = QVKDC_MIXED_QEFF_LENET_HEA(qeff_weights_param = weights_qeff_method, num_classes_param = NUM_CLASSES, dim_lenet_out_param = DIM_LENET_OUT, input_dim_param = INPUT_DIM, var_hea_ansatz_size_param = HEA_ANSATZ_SIZE, num_layers_hea_param = NUM_LAYERS_HEA, gamma=GAMMA, epochs_param = EPOCHS)
qvkdc_hea_lenet.fit(X_train, y_train_oh, batch_size=32)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 20)        520       
                                                                 
 average_pooling2d_8 (Avera  (None, 14, 14, 20)        0         
 gePooling2D)                                                    
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 50)        25050     
                                                                 
 average_pooling2d_9 (Avera  (None, 7, 7, 50)          0         
 gePooling2D)                                                    
                                                                 
 flatten_4 (Flatten)         (None, 2450)              0         
                                                                 
 dense_8 (Dense)             (None, 84)               

AssertionError: Concurrent access?

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = qvkdc_hea_lenet.predict(X_test)
accuracy_score(y_test, np.argmax(y_pred, axis=1))

9 16 8 4 4
313/313 [==============================] - 37s 37ms/step


0.9902